# Fluxo Geral
1. Carregamento e Exploração dos Dados
2. Pré-Processamento dos Dados
3. Divisão dos Dados (Treino, Validação e Teste)
4. Treinamento de Modelos
5. Avaliação de Modelos
6. Ajustes e Melhorias
7. Implantação da IA

## Passo 1: Carregamento e Exploração dos Dados

In [ ]:
# Carregar os dados
import pandas as pd

dataset_path = "../data/raw/dataset_transacoes.csv"
df = pd.read_csv(dataset_path)

# Visualizar as primeiras linhas
print(df.head())

# Informações gerais sobre os dados
print(df.info())

# Resumo estatístico
print(df.describe())



## Passo 2: Análise Exploratória


### Distribuição das Classes

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(x="classe", data=df)
plt.title("Distribuição das Classes (Fraude vs Não Fraude)")
plt.show()


### Correlação entre Atributos

In [ ]:
# Identificar colunas categóricas
categorical_cols = ["localizacao", "tipo_estabelecimento", "dia_semana", "categoria_despesa", "genero_titular", "historico_pagamento"]

# Codificar variáveis categóricas usando One-Hot Encoding
df_encoded = pd.get_dummies(df, columns=categorical_cols)

# Gerar a matriz de correlação para o DataFrame codificado
plt.figure(figsize=(12, 8))
sns.heatmap(df_encoded.corr(), annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Matriz de Correlação (Atributos Codificados)")
plt.show()


In [ ]:
# Selecionar apenas colunas numéricas
numerical_df = df.select_dtypes(include=["number"])

# Gerar a matriz de correlação
plt.figure(figsize=(12, 8))
sns.heatmap(numerical_df.corr(), annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Matriz de Correlação (Atributos Numéricos)")
plt.show()


### 3. Visualizar Correlações Apenas para Colunas de Interesse

Se você quer ver correlações apenas entre algumas colunas específicas:

In [ ]:

# Selecionar colunas específicas
cols_of_interest = ["valor_transacao", "limite_credito", "saldo_atual", "classe"]
plt.figure(figsize=(8, 6))
sns.heatmap(df[cols_of_interest].corr(), annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Correlação Entre Colunas Selecionadas")
plt.show()


### Normalização de Dados Numéricos
Adicionaremos a normalização para as colunas numéricas, garantindo que todos os atributos estejam na mesma escala:

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Selecionar colunas numéricas para normalização
numerical_cols = [
    "valor_transacao", "limite_credito", "saldo_atual", "valor_medio_transacoes",
    "utilizacao_credito", "tempo_desde_ultima_transacao"
]

# Criar o escalador
scaler = MinMaxScaler()

# Normalizar os dados numéricos
df_encoded[numerical_cols] = scaler.fit_transform(df_encoded[numerical_cols])

# Verificar as primeiras linhas após a normalização
print(df_encoded.head())


### Divisão do Dataset
Dividiremos os dados em conjuntos de treino, validação e teste:

In [ ]:
from sklearn.model_selection import train_test_split

# Separar atributos (X) e classe (y)
X = df_encoded.drop("classe", axis=1)
y = df_encoded["classe"]

# Divisão inicial: 70% treino, 30% temporário
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Divisão secundária: 15% validação, 15% teste
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# Confirmar os tamanhos dos conjuntos
print(f"Tamanho do conjunto de treino: {X_train.shape}")
print(f"Tamanho do conjunto de validação: {X_val.shape}")
print(f"Tamanho do conjunto de teste: {X_test.shape}")


### Treinamento Inicial com Random Forest
Usaremos a Random Forest como primeiro modelo para classificar as transações:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Criar o modelo
model = RandomForestClassifier(random_state=42)

# Treinar o modelo
model.fit(X_train, y_train)

# Fazer previsões no conjunto de validação
y_pred = model.predict(X_val)

# Avaliar o modelo
print("Relatório de Classificação:\n", classification_report(y_val, y_pred))
print("Matriz de Confusão:\n", confusion_matrix(y_val, y_pred))


## Análise do Problema
### Matriz de Confusão:

O modelo previu corretamente todas as instâncias da classe 0 (144), mas não conseguiu prever nenhuma instância da classe 1 (todas foram classificadas como 0).
### Métricas:

- Precision (1): Zero porque o modelo não fez nenhuma previsão correta da classe 1.
- Recall (1): Zero porque o modelo não conseguiu identificar nenhuma instância real da classe 1.
Causa:

O dataset está altamente desbalanceado (cerca de 5% de fraudes). Modelos de aprendizado de máquina padrão tendem a favorecer a classe majoritária.

## Soluções
Vamos abordar o problema de desbalanceamento com algumas técnicas:

1. Reamostragem do Dataset
Oversampling (Aumentar a classe minoritária):
Usaremos SMOTE para gerar amostras sintéticas da classe 1.

In [ ]:
from imblearn.over_sampling import SMOTE

# Criar o objeto SMOTE
smote = SMOTE(random_state=42)

# Aplicar oversampling nos dados de treino
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Verificar a nova distribuição
print("Distribuição após SMOTE:")
print(y_train_balanced.value_counts())


In [ ]:
# Criar o modelo Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Treinar o modelo com os dados balanceados
model_balanced = RandomForestClassifier(random_state=42)
model_balanced.fit(X_train_balanced, y_train_balanced)

# Fazer previsões no conjunto de validação
y_pred_balanced = model_balanced.predict(X_val)

# Avaliar o modelo
print("Relatório de Classificação (Dados Balanceados):\n", classification_report(y_val, y_pred_balanced))
print("Matriz de Confusão (Dados Balanceados):\n", confusion_matrix(y_val, y_pred_balanced))


In [ ]:
# Aplicar SMOTE também no conjunto de validação
X_val_balanced, y_val_balanced = smote.fit_resample(X_val, y_val)

# Reavaliar o modelo com os dados balanceados no conjunto de validação
y_pred_balanced_val = model_balanced.predict(X_val_balanced)

# Avaliar o modelo novamente
print("Relatório de Classificação (Validação Balanceada):\n", classification_report(y_val_balanced, y_pred_balanced_val))
print("Matriz de Confusão (Validação Balanceada):\n", confusion_matrix(y_val_balanced, y_pred_balanced_val))


In [ ]:
from xgboost import XGBClassifier

# Criar o modelo XGBoost
xgb_model = XGBClassifier(random_state=42, scale_pos_weight=len(y_train_balanced) / sum(y_train_balanced == 1))

# Treinar o modelo com os dados balanceados
xgb_model.fit(X_train_balanced, y_train_balanced)

# Fazer previsões no conjunto de validação
y_pred_xgb = xgb_model.predict(X_val)

# Avaliar o modelo
print("Relatório de Classificação (XGBoost):\n", classification_report(y_val, y_pred_xgb))
print("Matriz de Confusão (XGBoost):\n", confusion_matrix(y_val, y_pred_xgb))
